Install and import PyPDF2. Make a PDF Reader object, assign PDF text into string variable for manipulation. Import OS to get a list of reports avaiable.

In [126]:
import PyPDF2 as PDF2
import os

report_path = './Reports/'
total_reports_available = os.listdir(report_path)

In [127]:
import re
yt_report_pdf = open('./Reports/'+ total_reports_available[len(total_reports_available)-1],'rb')
# I want to find a way to just use the most recent report, I'm not sure how they get organized in the folder yet though.
# yt_report_pdf = open('./Reports/Walmart-YouTube-Threat-Report-2023-11-27 17:51:31.pdf','rb')

yt_reader = PDF2.PdfReader(yt_report_pdf)

num_of_pages = len(yt_reader.pages)
clean_comment_list = []
total_comment_list = []

for yt_page in range(num_of_pages):
    extracted_page = yt_reader.pages[yt_page].extract_text()
    clean_comment_list.append(extracted_page[extracted_page.find('Comments')+9::].replace('\t', '').replace('-', '').replace("\"",'').replace('\'', '').split('\n'))
    
for index in range(len(clean_comment_list)):
    if len(clean_comment_list[index]) < 2:
        continue
    for word in range(len(clean_comment_list[index])):
        if clean_comment_list[index][word] != '':
            total_comment_list.append(clean_comment_list[index][word])

yt_report_pdf.close()

In [128]:
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
convert_str = ''

for index in total_comment_list:
    convert_str += index.lower()+ '.'
    
blob = TextBlob(convert_str, analyzer=NaiveBayesAnalyzer())
print(blob.sentiment)

# OUTPUT: Sentiment(classification='pos', p_pos=0.9794064737603723, p_neg=0.02059352623962015)

Sentiment(classification='pos', p_pos=0.9794064737603723, p_neg=0.02059352623962015)


In [129]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
print(sia.polarity_scores(convert_str))
# OUTPUT: {'neg': 0.048, 'neu': 0.8, 'pos': 0.153, 'compound': 0.9976}

{'neg': 0.048, 'neu': 0.8, 'pos': 0.153, 'compound': 0.9976}


In [130]:
from statistics import mean
import nltk
text = convert_str

def is_positive() :
    scores = [
        sia.polarity_scores(sentence)["compound"] for sentence in nltk.sent_tokenize(text)
    ]
    return mean(scores)

print(is_positive() )
# OUTPUT: True/ 0.2124184210526316


0.2124184210526316


In [131]:

def get_general_score():
    if(is_positive() and blob.sentiment.classification == 'pos'):
        return f'Sentiment Scores: NLTK SentimentIntensityAnalyzer: {is_positive()}...TextBlob NaiveBayesAnalyzer: {blob.sentiment.classification}...Overall: Positive'
    elif is_positive() != 0:
        return f'Sentiment Scores: NLTK SentimentIntensityAnalyzer: {is_positive()}...TextBlob NaiveBayesAnalyzer: {blob.sentiment.classification}...Overall: Negative'
    else:
        return f'Sentiment Scores: NLTK SentimentIntensityAnalyzer: {is_positive()}...TextBlob NaiveBayesAnalyzer: {blob.sentiment.classification}...Overall: Neutral'
print(get_general_score())

Sentiment Scores: NLTK SentimentIntensityAnalyzer: 0.2124184210526316...TextBlob NaiveBayesAnalyzer: pos...Overall: Positive


I added TextBlob NaiveBayesAnalyzer (I'm not sure if that was the default), and NLTK SentimentIntensityAnalyzer. I feel like I need to format/clean up the data some more to make it really accurate but this seems to be an okay starting spot. I'll keep experimenting on where to go after this; but I was thinking of taking one or two reports, formatting the data and training a model using that data. I want to find a way to just use the most recent report, I'm not sure how they get organized in the folder yet though. 